## Zadania: LangChain 1.X.X+ – nowa składnia: create_agent, messages, structured output, pamięć, middleware, streaming, MCP

Ten notebook jest **dodatkowym uzupełnieniem** do kursu – pokazuje kilka kluczowych elementów z nowej składni LangChain (1.x), w szczególności API wokół `create_agent`.

### Instalacja bibliotek

In [ ]:
!pip install -q langchain python-dotenv langchain_mcp_adapters fastmcp

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

Zadanie 1. create_agent

Zadanie: Zmodyfikuj poniższy kod tak, aby wykorzystane narzędzie, zamiast oceniac miasto, zwracało przepis na wybrane danie. Możesz wykorzystać slownik i dodać przepisy dla kilku dań. Jeżeli danie nie występuje w kluczach słownika, zwróć przepis na "wodę na herbatę'.

In [2]:
from langchain.agents import create_agent

def rate_city(city: str) -> str:
    """Rate the city."""
    return f"{city} is the best place in the world!"

agent = create_agent(
    model="gpt-5-mini",
    tools=[rate_city],
    system_prompt="You are a helpful assistant",
)

result = agent.invoke({"messages": [{"role": "user", "content": "Is Poznań a nice city?."}]})

last_msg = result["messages"][-1]
print(last_msg.content)


Short answer: yes — Poznań is generally considered a very nice city, especially for history, culture, students and a relaxed, lived-in Polish city vibe.

Reasons people like Poznań
- Beautiful Old Town (Stary Rynek) with the Renaissance Town Hall and the famous mechanical goats that butt heads at noon.
- Rich history: Cathedral Island (Ostrów Tumski), churches, and museums (National Museum, Archeological Museum).
- Green space: Lake Malta (recreation, rowing, walking), Citadel Park (large park, monuments, views).
- Vibrant cultural scene: Malta Festival, theatres, concerts and active contemporary arts.
- Student energy: Adam Mickiewicz University gives the city a lively cafe/bar/nightlife scene and affordable cultural offerings.
- Food and local specialties: Poznań’s St. Martin’s croissant (rogale świętomarcińskie), good cafes and craft beer.
- Good transport links: international airport (Poznań–Ławica), rail connections to Warsaw, Berlin and other cities; compact city centre that's ea

Zadanie 2.  Structured output – `response_format` w `create_agent`

W poniższym przykładzie dodaj adres zamieszkania osoby w promptcie. Dodaj też parametr `address` w modelu pydantic.


In [4]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address")
    phone: str = Field(description="The phone number")

agent = create_agent(
    model="gpt-5-mini",
    response_format=ContactInfo,
)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}
    ]
})

structured = result["structured_response"]
print(structured)
print(type(structured))


name='John Doe' email='john@example.com' phone='(555) 123-4567'
<class '__main__.ContactInfo'>


Zadanie 3. Short‑term memory

W poniższym przykładzie najpierw poinformuj model o tym jaka jest dziś pogoda. Natępnie zapytaj model jaka jest dziś pogoda.


In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_agent(
    model="gpt-5-mini",
    checkpointer=checkpointer,
)

config = {"configurable": {"thread_id": "demo-thread-1"}}

agent.invoke({"messages": [{"role": "user", "content": "Hi! My name is Michał."}]}, config=config)
result = agent.invoke({"messages": [{"role": "user", "content": "What is my name?"}]}, config=config)
print(result["messages"][-1].content)


Your name is Michał.


Zadanie 4. Middleware -Guardrails – PII middleware

Podmień wrażliwe dane w poniższym przyklaadzie i uruchom ponownie kod.

In [8]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware

def echo(text: str) -> str:
    """Print text."""
    return text

agent = create_agent(
    model="gpt-5-mini",
    tools=[echo],
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
            apply_to_input=True,
        ),
    ],
)

out = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Extract information from text: My email is john@example.com and card is 5105-1051-0510-5100"
    }]
})
print(out["messages"][-1].content)


Extracted fields:
- Email: [REDACTED_EMAIL]
- Card (masked): ****-****-****-5100
- Card last 4 digits: 5100

Note: The email appears to be already redacted in the input. If you want a different output format (JSON, CSV, etc.), tell me which.


Zadanie 5.  Minimalny MCP server (FastMCP)
Zapisz poniższy kod jako `math_server.py` w tym samym katalogu co ten notebook.


```python
from fastmcp import FastMCP

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    "Add two numbers"
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    "Multiply two numbers"
    return a * b

if __name__ == "__main__":
    mcp.run(transport="stdio")
```

Następnie uruchom ponizszy kod. Spróbuj dodac kolejny tool MCP realizujący potęgowanie.

In [1]:
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

async def demo_mcp():
    client = MultiServerMCPClient(
        {
            "math": {
                "transport": "stdio",
                "command": "python",
                "args": ["math_server.py"],
            },
        }
    )
    tools = await client.get_tools()
    agent = create_agent("gpt-5-mini", tools)

    r1 = await agent.ainvoke({"messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]})
    print(r1["messages"][-1].content)

# Odkomentuj poniżej, aby uruchomić
# asyncio.run(demo_mcp())